# Phase 8 — Impact du risque climatique sur le risque de crédit

But : relier l’exposition climatique (FEMA EAL par État) au portefeuille LendingClub
pour mesurer l’association entre climat et risque de crédit.

On produit 3 analyses :

1) Exposition portefeuille par niveau climatique
2) Taux de défaut observé par niveau climatique
3) PD prédite moyenne par niveau climatique (recalcul PD avec le modèle logistique L2)
4) Calcul de l'Expected Credit Loss (ECL) pour chaque prêt et pour le portefeuille
5. Création de KPI de gestion du portefeuille


In [21]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
loans = pd.read_csv("data/interim/lendingclub_light.csv", low_memory=False)
climate = pd.read_csv("../reports/tableau_exports/07_climate_risk_by_state.csv")

print("Loans:", loans.shape)
print("Climate:", climate.shape)

loans.head(3)


Loans: (250000, 8)
Climate: (56, 3)


,loan_amnt,term,int_rate,grade,annual_inc,loan_status,addr_state,dti
0,20000.0,36 months,13.99,C,65000.0,Fully Paid,CA,13.68
1,7000.0,36 months,9.16,B,35000.0,Fully Paid,TX,22.39
2,20000.0,36 months,8.67,B,90000.0,Fully Paid,UT,29.14


In [67]:
#Creation de la variable cible

loans["default_flag"] = loans["loan_status"].astype(str).str.lower().apply(
    lambda x: 1 if x in ["charged off", "default"] else 0)
loans["default_flag"].value_counts()


default_flag
0    200103
1     49897
Name: count, dtype: int64

In [66]:
loans["default_flag"].value_counts(normalize = True)

default_flag
0    0.800412
1    0.199588
Name: proportion, dtype: float64

In [7]:
print("Unique states in loans:", loans["addr_state"].nunique())
print("Unique states in climate:", climate["STATEABBRV"].nunique())

loans["addr_state"].head(10), climate["STATEABBRV"].head(10)

Unique states in loans: 51
Unique states in climate: 56


(0    CA
 1    TX
 2    UT
 3    IL
 4    WI
 5    NV
 6    KY
 7    MS
 8    CA
 9    CA
 Name: addr_state, dtype: object,
 0    AL
 1    AK
 2    AZ
 3    AR
 4    CA
 5    CO
 6    CT
 7    DE
 8    DC
 9    FL
 Name: STATEABBRV, dtype: object)

In [68]:
#jointure des deux dataset

df = loans.merge(
    climate,
    left_on="addr_state",
    right_on="STATEABBRV",
    how="left"
)

print("Merged df shape:", df.shape)
df[["addr_state", "STATEABBRV", "EAL_SCORE", "climate_risk_level"]].head(10)

Merged df shape: (250000, 12)


,addr_state,STATEABBRV,EAL_SCORE,climate_risk_level
0,CA,CA,100.000000,Very High
1,TX,TX,96.428571,Very High
2,UT,UT,64.285714,High
3,IL,IL,85.714286,Very High
4,WI,WI,41.071429,Low
5,NV,NV,57.142857,Medium
6,KY,KY,55.357143,Medium
7,MS,MS,71.428571,High
8,CA,CA,100.000000,Very High
9,CA,CA,100.000000,Very High


In [69]:
#controle des valeurs manquantes 
missing_rate = df["climate_risk_level"].isna().mean()
print("Missing climate_risk_level rate:", missing_rate)

missing_states = df.loc[df["climate_risk_level"].isna(), "addr_state"].value_counts().head(20)
print("Missing states (top):")
print(missing_states)



Missing climate_risk_level rate: 0.0
Missing states (top):
Series([], Name: count, dtype: int64)


In [14]:
qc = pd.DataFrame([{
    "missing_rate_climate_join": float(missing_rate),
    "n_loans": int(df.shape[0]),
    "n_states_loans": int(loans["addr_state"].nunique()),
    "n_states_climate": int(climate["STATEABBRV"].nunique())
}])

qc.to_csv("../reports/tableau_exports/08_merge_quality_check.csv", index=False)
print("Export OK: 08_merge_quality_check.csv")


Export OK: 08_merge_quality_check.csv


#### Exposition (volume + montants)
___

In [72]:
if "id" not in df.columns:
    df["id"] = np.arange(len(df))

exposure = df.groupby("climate_risk_level").agg(
    n_loans=("id", "count"),
    total_loan_amnt=("loan_amnt", "sum"),
    avg_loan_amnt=("loan_amnt", "mean")
).reset_index()

exposure


,climate_risk_level,n_loans,total_loan_amnt,avg_loan_amnt
0,High,55367,7.917874e+08,14300.710712
1,Low,23733,3.422375e+08,14420.322968
2,Medium,45322,6.344340e+08,13998.366687
3,Very High,119757,1.750358e+09,14615.916815
4,Very Low,5821,8.286168e+07,14234.955334


#### Exposition du portefeuille par risque climatique

**Répartition des volumes**

`Very High`

119 757 prêts (catégorie dominante)
1,75 Md de montant total
→ Le portefeuille est fortement exposé aux zones à risque climatique très élevé.

___
`High et Medium`

Volumes significatifs (55k et 45k prêts)
Ensemble, ils représentent une part majeure du portefeuille.

___
`Low et Very Low`

Volumes plus faibles, surtout Very Low (5 821 prêts)
Exposition financière limitée.

**Montant moyen des prêts**

Relativement homogène entre les catégories (~14k–14,6k).
L’exposition au risque climatique provient donc du nombre de prêts, pas de montants unitaires plus élevés.

**Conclusion**

Le portefeuille est structurellement concentré sur des zones à risque climatique élevé à très élevé, ce qui constitue un enjeu stratégique de risque à moyen/long terme.

In [73]:

exposure.to_csv("../reports/tableau_exports/08_portfolio_exposure_by_climate.csv", index=False)
print("Export OK: 08_portfolio_exposure_by_climate.csv")

Export OK: 08_portfolio_exposure_by_climate.csv


#### Taux de défaut observé par niveau climat
___

In [74]:
default_by_climate = df.groupby("climate_risk_level").agg(
    n_loans=("default_flag", "size"),
    default_rate=("default_flag", "mean")
).reset_index()

default_by_climate


,climate_risk_level,n_loans,default_rate
0,High,55367,0.212455
1,Low,23733,0.187966
2,Medium,45322,0.201227
3,Very High,119757,0.197174
4,Very Low,5821,0.161484


#### Taux de défaut par risque climatique

**Lecture des taux de défaut**

`Very Low` : 16,1 % → le plus faible
`Low` : 18,8 %
`Medium` : 20,1 %
`High` : 21,2 % → le plus élevé
`Very High` : 19,7 %


**Relation globalement croissante**
 
Le taux de défaut augmente avec le risque climatique, surtout de Very Low à High.

**Cas particulier du** `“Very High”`

- Taux de défaut inférieur à “High”
- Mais volume massif de prêts
  
→ En valeur absolue, c’est la plus grande source de pertes potentielles.

**Effet volume vs effet taux**

- `High` : risque individuel plus élevé
`Very High` : risque systémique lié à la concentration

**Conclusion**

Le risque climatique est corrélé au défaut, mais non linéaire.
Le niveau Very High est critique non pas par son taux, mais par son poids dans le portefeuille.

In [18]:
default_by_climate.to_csv("../reports/tableau_exports/08_default_rate_by_climate.csv", index=False)
print("Export OK: 08_default_rate_by_climate.csv")


Export OK: 08_default_rate_by_climate.csv


#### Analyse 3 : PD prédite moyenne par niveau climat (recalcul PD)
___

##### Charger X/y 


In [23]:
X = pd.read_csv("../data/processed/X_features.csv")
y = pd.read_csv("../data/processed/y_target.csv").squeeze()

##### réentrainer le modèle (baseline L2)

In [75]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("First columns:", list(X.columns[:8]))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


X shape: (250000, 10)
y shape: (250000,)
First columns: ['loan_amnt', 'term_months', 'int_rate', 'annual_inc_clean', 'grade_B', 'grade_C', 'grade_D', 'grade_E']


##### Standardisation

In [57]:
scaler = StandardScaler()

# on garde les noms de colonnes après scaling
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_test.columns,
    index=X_test.index
)


##### Model avec regularisation

In [59]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)

print("Model trained (L2).")

X_train_scaled shape: (200000, 10)
X_test_scaled shape: (50000, 10)
Model trained (L2).


##### Refaire les features Engineering  sur df

In [76]:
# term_months
df["term_months"] = df["term"].astype(str).str.extract(r"(\d+)").astype(float)

# annual_inc_clean (cap p99)
p99 = df["annual_inc"].quantile(0.99)
df["annual_inc_clean"] = df["annual_inc"].clip(lower=0, upper=p99)

# grade dummies
df_model = pd.get_dummies(df, columns=["grade"], drop_first=True)

print("df_model created. shape:", df_model.shape)


df_model created. shape: (250000, 20)


#### PD prédite


In [78]:
model = LogisticRegression(penalty="l2", C=1.0, solver="lbfgs", max_iter=1000)
model.fit(X_train_scaled, y_train)

print("Model trained")
print("Example proba (first 5):", model.predict_proba(X_test_scaled)[:5, 1])


Model trained
Example proba (first 5): [0.23807355 0.30034548 0.37049702 0.20837551 0.04983957]


In [79]:
# Ajouter colonnes manquantes
for col in X.columns:
    if col not in df_model.columns:
        df_model[col] = 0

# Garder seulement les colonnes attendues, dans le bon ordre
X_portfolio = df_model[X.columns].copy()

print("X_portfolio shape:", X_portfolio.shape)
print("Columns aligned with X:", list(X_portfolio.columns) == list(X.columns))



X_portfolio shape: (250000, 10)
Columns aligned with X: True


In [81]:
#Scaling portefeuille + calcul PD 

X_portfolio_scaled = pd.DataFrame(
    scaler.transform(X_portfolio),
    columns=X_portfolio.columns,
    index=X_portfolio.index
)

df["pd_pred"] = model.predict_proba(X_portfolio_scaled)[:, 1]

print("pd_pred created")
df["pd_pred"].describe()


pd_pred created


count    250000.000000
mean          0.199642
std           0.112544
min           0.023986
25%           0.121176
50%           0.186398
75%           0.272819
max           0.661063
Name: pd_pred, dtype: float64

les probabilités de défaut sont comprises entre 0,02 et 0,66, avec une moyenne autour de 0,20

#### PD moyenne par niveau climat

In [82]:
pd_by_climate = df.groupby("climate_risk_level").agg(
    n_loans=("id", "count"),
    pd_mean=("pd_pred", "mean")
).reset_index()

pd_by_climate


,climate_risk_level,n_loans,pd_mean
0,High,55367,0.202919
1,Low,23733,0.196321
2,Medium,45322,0.203038
3,Very High,119757,0.197514
4,Very Low,5821,0.199359


#### Les zones à risque "High" et "Medium" ont une PD plus élevée
 
Ce qui indique une possible association entre le risque climatique élevé et un risque de défaut plus élevé.

`High` : 20.29% de PD

`Medium` : 20.30% de PD

Les zones à très haut risque climatique (High, Medium) montrent une probabilité de défaut plus élevée par rapport aux autres zones moins exposées.

Ces zones sont potentiellement plus sensibles aux risques physiques, ce qui peut impacter la stabilité financière des emprunteurs.

#### Les zones "Very Low" ont la PD la plus faible

Cette zone a la plus faible probabilité de défaut (19.94%).

`Very Low` : 19.94% de PD

les zones avec un risque climatique très faible sont souvent plus stables financièrement, et les emprunteurs sont moins affectés par des événements climatiques sévères.

#### Les zones "Very High" montrent un résultat moins élevé que prévu

`Very High` : 19.75% de PD

Bien que Very High représente un risque climatique physique très élevé, la PD n’est pas aussi élevée que celle de High ou Medium.

Cela suggère que d’autres facteurs (comme la notation de crédit interne, les revenus, la géographie, etc.) peuvent nuancer l’effet du climat sur le risque de crédit.

#### Zones à faible risque climatique

`Low` : 19.63% de PD

les zones avec un faible risque climatique montrent une PD légèrement inférieure, ce qui confirme que le climat a une influence mais pas déterminante sur le défaut de paiement. D'autres variables (ex : revenus, crédit) jouent un rôle important.

#### Zones à risque "Very Low" et "Very High" avec une PD similaire

`Very Low` : 19.94%

`Very High` : 19.75%

Cela n’indique pas un lien clair entre les deux à ce niveau.
Il est possible que d’autres variables aient un impact plus fort que le climat, et que le climat, dans ce cas, soit un facteur secondaire.

**CONCLUSIONS**

Association visible : Les zones à risque climatique élevé (High/Medium) montrent une PD plus élevée. Cela peut indiquer que les événements climatiques affectent indirectement la capacité de remboursement des emprunteurs.

Variation faible entre les extrêmes : Les zones avec un très faible risque climatique (Very Low) et un très haut risque climatique (Very High) ont une PD très similaire, ce qui suggère que d’autres facteurs (comme l’activité économique, la notation de crédit, etc.) influencent davantage la probabilité de défaut.

Anomalie dans les `"Very High"` : Pourquoi ces zones ne montrent-elles pas une PD plus élevée ? Cela pourrait être dû à des facteurs compensatoires comme des emprunteurs mieux notés ou un revenu plus élevé. Une analyse plus poussée sur les facteurs de crédit (revenu, historique) serait utile.

In [83]:
pd_by_climate.to_csv("../reports/tableau_exports/08_pd_mean_by_climate.csv", index=False)
print("Export OK: reports/tableau_exports/08_pd_mean_by_climate.csv")


Export OK: reports/tableau_exports/08_pd_mean_by_climate.csv


### Interprétation des résultats

Cette analyse mesure l’association entre le **risque climatique physique** et la **probabilité de défaut (PD)** dans le portefeuille de prêts. Les principaux résultats sont les suivants :

1. **Exposition par niveau climatique** :
   - Les prêts sont majoritairement concentrés dans les zones **Very High** et **High**. Cela montre que le portefeuille est **plus exposé aux risques climatiques importants**, avec une **concentration des volumes dans les zones vulnérables**.

2. **Taux de défaut observé** :
   - Les zones avec **high** et **medium** niveaux de risque climatique montrent une **PD légèrement plus élevée**. Cela suggère une **correlation positive** entre l'exposition au risque climatique et le défaut de paiement, bien que l'impact soit limité.
   - Les zones **Very Low** et **Low** ont des taux de défaut plus faibles, ce qui peut être dû à une plus grande stabilité économique dans ces régions.

3. **PD moyenne par niveau climatique** :
   - La **PD moyenne prédite** montre que les **zones à haut risque climatique (High, Medium)** ont une PD plus élevée par rapport aux zones à faible risque (Very Low, Low).
   - Cependant, il est **notable que les zones à risque très élevé (Very High)** ne présentent pas une PD significativement plus élevée que les autres catégories à risque élevé. Cela pourrait indiquer que **d’autres facteurs de risque (notation interne, revenu, etc.) compensent l’impact du climat sur la probabilité de défaut**.

---

### Conclusion:

1. **Association statistique observée** : Une corrélation existe entre l’exposition climatique physique et la PD, mais cette relation n’est **pas aussi forte qu’espéré** dans certaines zones, comme le **Very High**.
2. **Facteurs compensateurs** : Il semble que **d'autres variables** (comme la **notation de crédit interne**, les **revenus des emprunteurs**, et d’autres critères économiques) jouent un rôle plus important dans certaines zones.
3. **Next step** : Il serait pertinent de **segmenter davantage** par critères internes (grade, revenu, etc.) pour voir si l’impact climatique est plus évident au sein de certains groupes de profils (emprunteurs avec faible notation par exemple).

Cette analyse **prépare le terrain** pour la Phase suivante (Phase 9 — Tableau Public), où l’on analysera plus finement l’impact du climat sur le portefeuille à travers des visualisations interactives.
